# Michelin Restaurants in Italy: Data Exploration and Search Engine


# RQ1. Data Collection


I started by collecting the restaurants' URLs from the Michelin Guide's website for Italy. To do this, I used a web scraper built with requests and BeautifulSoup. The scraper iteratively constructs paginated URLs (/page/1, /page/2, ecc). I considered only links that contains /restaurant/ in their url, and I stored them in a Python set (in order to avoid duplicates).

## 1.1. Get the List of Michelin Restaurants


In [ ]:
# Importing necessary libraries

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os
import shutil
import re
import pandas as pd

# Set the base URL, the Michelin guide site for restaurants in Italy
base_url = "https://guide.michelin.com/en/it/restaurants"
all_urls = set()  # Inizialize the set to store unique URLs
page_number = 1   # Initialize the page number

while True:
  # Construct the URL for the current page
  paginated_url = f"{base_url}/page/{page_number}"
  response = requests.get(paginated_url)

  # Check if the request was successful (status code 200) or not
  if response.status_code != 200:
    print(f"Stopping at page {page_number}, status code: {response.status_code}")
    break  # If the request fails, stop (no more pages to fetch)

  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.text, "html.parser")

  # Find all <a> tags with a href attribute (hyperlinks)
  links = soup.find_all("a", href=True)

  # Filter links to include only the ones containing "restaurant" in theirs URLs
  filtered_links = list(filter(lambda link: "/restaurant/" in link["href"], links))

  # Convert relative URLs to absolute URLs and add them to the set
  for link in filtered_links:
    all_urls.add(urljoin(base_url, link["href"]))

  # Check for the end of the page
  if not filtered_links:
    print("No more restaurants on this page, stopping pagination")
    break

  # Increment the page number
  page_number += 1

# Output the results
print(f"Collected {len(all_urls)} unique restaurant links")
for url in all_urls:
    print(url)

Stopping at page 101, status code: 404
Collected 1981 unique restaurant links
https://guide.michelin.com/en/emilia-romagna/milano-marittima/restaurant/terrazza-bartolini
https://guide.michelin.com/en/friuli-venezia-giulia/tavagnacco/restaurant/al-grop
https://guide.michelin.com/en/veneto/rubano/restaurant/le-calandre
https://guide.michelin.com/en/emilia-romagna/campogalliano/restaurant/magnagallo
https://guide.michelin.com/en/veneto/asolo/restaurant/locanda-baggio
https://guide.michelin.com/en/lazio/latina/restaurant/il-funghetto
https://guide.michelin.com/en/trentino-alto-adige/corvara-in-badia/restaurant/ladinia
https://guide.michelin.com/en/veneto/mestre/restaurant/all-ombra-del-gabbiano
https://guide.michelin.com/en/lombardia/legnano/restaurant/koine
https://guide.michelin.com/en/emilia-romagna/modena/restaurant/trattoria-pomposa-al-re-gras
https://guide.michelin.com/en/toscana/firenze/restaurant/degusteria-italiana
https://guide.michelin.com/en/emilia-romagna/castelnovo-di-sotto/r

Once we have a list of restaurant URLs, the next step is to fetch and save the HTML content of each page. The URLs are loaded from **all_links.txt** for processing. The script sends the requests to retrieve the HTML content of each URL, and the fetched HTML content is saved immediately to a local directory (**michelin_restaurant_html**).

The saved HTML files are organized into subfolders based on pages. Files are moved from the main output directory to organized_michelin_html, and they are grouped into subfolders (Page_1, Page_2, ecc). Each subfolder contains up to 20 HTML files, as the official website's pagination.

## 1.2. Crawl Michelin restaurant pages

In [ ]:
# Save the collected links to a file
with open("all_links.txt", "w") as f:

    # Write each URL to a new line in the file
    f.writelines(f"{url}\n" for url in all_urls)

# Read URLs from the file
with open("all_links.txt", "r") as file:

    # Split lines into a list of URLs
    urls_list = file.read().splitlines()

print(f"Loaded {len(urls_list)} URLs.")
print(urls_list)

# Directory to save the HTML files
output_folder = "michelin_restaurant_html"
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Fetch and save HTML files
for i, url in enumerate(urls_list):
    try:
        # Send a request to fetch the HTML content
        response = requests.get(url)
        response.raise_for_status()  # Raise an error if the request fails

        # Save HTML content immediately to a file
        filename = f"{output_folder}/restaurant_{i+1}.html"
        with open(filename, "w", encoding="utf-8") as file:
            file.write(response.text)

        print(f"Successfully saved: {filename}")

    except requests.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

# Organize HTML files into subfolders
output_folder = "michelin_restaurant_html"    # Folder with the saved HTML files
organized_folder = "organized_michelin_html"  # Target folder for organized structure

# Create the main folder for organized HTML files if it doesn't exist
os.makedirs(organized_folder, exist_ok=True)

# Get a sorted list of all HTML files in the output folder
html_files = sorted(os.listdir(output_folder))
files_per_page = 20  # As the official website

# Loop to move files into page-based folders
for start_index in range(0, len(html_files), files_per_page):
    page_number = (start_index // files_per_page) + 1  # Calculate page number

    # Create a subfolder for each page
    page_folder = os.path.join(organized_folder, f"Page_{page_number}")
    os.makedirs(page_folder, exist_ok=True)  # Create a subfolder if it doesn't exist

    # Move files for the current page into the appropriate folder
    for file_name in html_files[start_index:start_index + files_per_page]:
        file_to_move = os.path.join(output_folder, file_name)
        destination_path = os.path.join(page_folder, file_name)
        if os.path.exists(file_to_move):
            shutil.move(file_to_move, destination_path)
        else:
            print(f"File {file_to_move} not found, skipping move.")

print("Files have been organized into folders based on pages.")

# Get list of subfolders and sort them numerically based on the page number
subfolders = [f.path for f in os.scandir(organized_folder) if f.is_dir()]
subfolders = sorted(subfolders, key=lambda x: int(re.search(r'\d+', x).group()))

# Print out the contents of each subfolder in ascending numerical order
for subfolder in subfolders:

    # Get a sorted list of files in the current subfolder
    file_list = sorted(os.listdir(subfolder))  # Sort files in each subfolder for order consistency
    print(f"{subfolder} contains {len(file_list)} files.")

Loaded 1981 URLs.
['https://guide.michelin.com/en/emilia-romagna/milano-marittima/restaurant/terrazza-bartolini', 'https://guide.michelin.com/en/friuli-venezia-giulia/tavagnacco/restaurant/al-grop', 'https://guide.michelin.com/en/veneto/rubano/restaurant/le-calandre', 'https://guide.michelin.com/en/emilia-romagna/campogalliano/restaurant/magnagallo', 'https://guide.michelin.com/en/veneto/asolo/restaurant/locanda-baggio', 'https://guide.michelin.com/en/lazio/latina/restaurant/il-funghetto', 'https://guide.michelin.com/en/trentino-alto-adige/corvara-in-badia/restaurant/ladinia', 'https://guide.michelin.com/en/veneto/mestre/restaurant/all-ombra-del-gabbiano', 'https://guide.michelin.com/en/lombardia/legnano/restaurant/koine', 'https://guide.michelin.com/en/emilia-romagna/modena/restaurant/trattoria-pomposa-al-re-gras', 'https://guide.michelin.com/en/toscana/firenze/restaurant/degusteria-italiana', 'https://guide.michelin.com/en/emilia-romagna/castelnovo-di-sotto/restaurant/poli-alla-stazi

## 1.3 Parse downloaded pages

In [ ]:
def extract_restaurant_data(html_content, url):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract restaurant's name (from the <h1> tag)
    name_tag = soup.find("h1")
    restaurant_name = name_tag.get_text(strip=True) if name_tag else ""

    # Extract restaurant's address (from the <div> class)
    address_tag = soup.find("div", class_="data-sheet__block--text")
    restaurant_address = address_tag.get_text(strip=True) if address_tag else ""

    # Split address into components
    address, city, postal_code, country = "", "", "", ""
    if restaurant_address:
        address_parts = [part.strip() for part in restaurant_address.split(",")]
        if len(address_parts) >= 4:
            # Extract address components if all four parts are present
            address = address_parts[0]
            city = ", ".join(address_parts[1:-2])
            postal_code = address_parts[-2].strip()
            country = address_parts[-1].strip()
        elif len(address_parts) == 3:
            # Case with three components (missing postal code)
            address = address_parts[0]
            city = address_parts[1].strip()
            postal_code = ""
            country = address_parts[2].strip()
        elif len(address_parts) == 2:
            # Case with only address and country
            address = address_parts[0]
            city = ""
            postal_code = ""
            country = address_parts[1].strip()

    # Extract restaurant's price range and restaurant's cuisine type (from the <div> class)
    price_range, cuisine_type = "", ""
    price_description_tags = soup.find_all("div", class_="data-sheet__block--text")
    if price_description_tags and len(price_description_tags) > 1:

        # Split content into price range and cuisine type
        content = price_description_tags[1].get_text().strip()
        content_parts = content.split('·')
        if len(content_parts) == 2:
            price_range = content_parts[0].strip()
            cuisine_type = content_parts[1].strip()
        elif len(content_parts) == 1:
            price_range = content_parts[0].strip()

    # Extract restaurant's description (from the <div> class)
    description_tag = soup.find("div", class_="data-sheet__description")
    restaurant_description = description_tag.get_text(strip=True) if description_tag else ""

    # Extract restaurant's services (from the <div> class)
    services = []
    services_container = soup.find("div", class_="restaurant-details__services")
    if services_container:
        services_tags = services_container.find_all("li")
        services = [tag.get_text(strip=True) for tag in services_tags] if services_tags else []

    # Extract credit card's information (from the <div> class)
    credit_cards = []
    credit_cards_container = soup.find("div", class_="list--card")
    if credit_cards_container:
        credit_card_tags = credit_cards_container.find_all("img")
        for tag in credit_card_tags:
            data_src = tag.get("data-src", "")
            if "amex" in data_src:
                credit_cards.append("Amex")
            elif "mastercard" in data_src:
                credit_cards.append("Mastercard")
            elif "visa" in data_src:
                credit_cards.append("Visa")

    # Extract restaurant's phone number (from the <div> class)
    phone_number = ""
    phone_container = soup.find("div", class_="collapse__block-item")
    if phone_container:
        phone_tag = phone_container.find("span", dir="ltr")
        phone_number = phone_tag.get_text(strip=True) if phone_tag else ""

    # Extract restaurant's website URL (from the <div> class)
    website = ""
    website_container = soup.find("div", class_="collapse__block-item link-item")
    if website_container:
        website_tag = website_container.find("a", href=True)
        website = website_tag["href"] if website_tag else ""

    # Return extracted data as a dictionary
    return {
        "restaurantName": restaurant_name or "",
        "address": restaurant_address or "",
        "city": city or "",
        "postalCode": postal_code or "",
        "country": country or "",
        "priceRange": price_range or "",
        "cuisineType": cuisine_type or "",
        "description": restaurant_description or "",
        "facilitiesServices": services if services else [],
        "creditCards": credit_cards if credit_cards else [],
        "phoneNumber": phone_number or "",
        "website": website or ""
    }

In [ ]:
def parse_all_html_files(input_folder, output_file):

    # List to store data for all restaurants
    all_restaurants_data = []

    # Walk through all subfolders and files within the input folder
    for dirpath, _, filenames in os.walk(input_folder):
        for file in sorted(filenames):
            if file.endswith(".html"):

                # Open and read the content of each HTML file
                file_path = os.path.join(dirpath, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    html_content = f.read()
                    url = f"http://example.com/{file}"
                    data = extract_restaurant_data(html_content, url)
                    all_restaurants_data.append(data)

    # Create a DataFrame
    df = pd.DataFrame(all_restaurants_data)

    # Save the DataFrame in a TSV file
    df.to_csv(output_file, sep="\t", index=False)
    print(f"Data saved to {output_file}")

# Define input and output paths
input_folder = "organized_michelin_html"  # Main directory containing all folders
output_file = "parsed_restaurants.tsv"
parse_all_html_files(input_folder, output_file)

Data saved to parsed_restaurants.tsv


# RQ2. Search Engine

## Importing

In [ ]:
df = pd.read_csv('cleaned_restaurants.csv')

df.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Castel Toblino,località Castel Toblino 1,Castel Toblino,38076,Italy,€€,"Modern Cuisine, Contemporary",Situated in a bucolic location in the Trentino...,"['Car park', 'Garden or park', 'Terrace']","['Amex', 'Mastercard', 'Visa']",+39 0461 864036,https://www.casteltoblino.com/
1,Su Gologone,Località Su Gologone,Oliena,8025,Italy,€€,"Sardinian, Traditional Cuisine",It is difficult to choose between the three de...,"['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Mastercard', 'Visa']",+39 0784 287512,https://www.sugologone.it/ristorante-tipico.html
2,Bàcaro Il Gusto,via Provinciale Nord 30,Fossò,30030,Italy,€€,Venetian,"A simple, modern restaurant where all the atte...","['Air conditioning', 'Wheelchair access']","['Amex', 'Mastercard', 'Visa']",+39 041 517 0035,https://www.ristorantebacaroilgusto.it
3,Osteria dello Strecciolo,via Indipendenza 2,Robbiate,23899,Italy,€€€,Italian Contemporary,"Situated in the centre of the village, this po...",[],"['Amex', 'Mastercard', 'Visa']",+39 039 928 1052,http://www.osteriadellostrecciolo.it
4,Trippa,via Giorgio Vasari 1,Milan,20135,Italy,€€,"Italian, Seasonal Cuisine","Simple, informal and with a slightly retro fee...","['Air conditioning', 'Terrace', 'Wheelchair ac...","['Amex', 'Mastercard', 'Visa']",+39 327 668 7908,https://www.trippamilano.it/


## 2.0 Preprocessing the text

To build an effective search engine for restaurant descriptions, we need to preprocess the text data to ensure consistency and improve search accuracy. Here’s what each step of the preprocessing does and why it is essential:

1. **Lowercasing**: All text is converted to lowercase to make searches case-insensitive. This ensures that words like "Vegan" and "vegan" are treated the same, avoiding mismatches due to capitalization.

2. **Removing Punctuation**: Punctuation marks (like commas, periods, and exclamation points) are removed, as they often do not contribute meaningfully in search contexts. This helps focus on the words themselves without interference from symbols.

3. **Removing Stopwords**: Commonly used words (like "and," "the," "in") are removed, as they generally do not add value in distinguishing between search results. Removing these words reduces noise in the data, allowing us to focus on the meaningful content of each description.

4. **Stemming**: Words are reduced to their root forms, meaning that variations of a word (such as "cooking," "cooked," and "cook") are treated as the same term. This normalization makes the data more consistent and compact, improving the effectiveness of search queries that include variations of keywords.

5. **Reassembling Cleaned Text**: After processing, the cleaned words are reassembled into a single, preprocessed string, ready for use in the search engine. This cleaned text better represents the core content of each description, making searches more accurate and consistent.

Through these steps, we make the text data cleaner and more uniform, which is essential for tasks like search or similarity matching by removing irrelevant elements and highlighting essential keywords.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Download the stopwords data (only needed once per environment)
nltk.download('stopwords')

# Initialize the stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_description(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize, remove stopwords, and apply stemming
    tokens = [stemmer.stem(word) for word in text.split() if word not in stop_words]

    # Join the tokens back into a cleaned text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Apply preprocessing to the 'description' column
df['processed_description'] = df['description'].apply(preprocess_description)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,processed_description
0,Castel Toblino,località Castel Toblino 1,Castel Toblino,38076,Italy,€€,"Modern Cuisine, Contemporary",Situated in a bucolic location in the Trentino...,"['Car park', 'Garden or park', 'Terrace']","['Amex', 'Mastercard', 'Visa']",+39 0461 864036,https://www.casteltoblino.com/,situat bucol locat trentino excel vino santo p...
1,Su Gologone,Località Su Gologone,Oliena,8025,Italy,€€,"Sardinian, Traditional Cuisine",It is difficult to choose between the three de...,"['Air conditioning', 'Car park', 'Garden or pa...","['Amex', 'Mastercard', 'Visa']",+39 0784 287512,https://www.sugologone.it/ristorante-tipico.html,difficult choos three delight room restaur one...
2,Bàcaro Il Gusto,via Provinciale Nord 30,Fossò,30030,Italy,€€,Venetian,"A simple, modern restaurant where all the atte...","['Air conditioning', 'Wheelchair access']","['Amex', 'Mastercard', 'Visa']",+39 041 517 0035,https://www.ristorantebacaroilgusto.it,simpl modern restaur attent focus extraordinar...
3,Osteria dello Strecciolo,via Indipendenza 2,Robbiate,23899,Italy,€€€,Italian Contemporary,"Situated in the centre of the village, this po...",[],"['Amex', 'Mastercard', 'Visa']",+39 039 928 1052,http://www.osteriadellostrecciolo.it,situat centr villag popular profession run res...
4,Trippa,via Giorgio Vasari 1,Milan,20135,Italy,€€,"Italian, Seasonal Cuisine","Simple, informal and with a slightly retro fee...","['Air conditioning', 'Terrace', 'Wheelchair ac...","['Amex', 'Mastercard', 'Visa']",+39 327 668 7908,https://www.trippamilano.it/,simpl inform slightli retro feel restaur serv ...


## 2.1 Conjuctive query

A **conjunctive query** is a search query that requires all specified terms to be present in the documents it retrieves. In the context of a search engine, this is often called an "AND search" because it returns only documents that contain **all** of the query terms.

For example, a conjunctive query for `"modern cuisine"` would only retrieve documents that contain both `"modern"` **and** `"cuisine"` in their content. This approach is useful when looking for highly specific results, but it can limit the number of results if one of the terms is uncommon or if the dataset contains few documents with all the terms.

To build a conjunctive search engine for restaurant descriptions, we will:
1. **Preprocess the Text**: Clean and standardize each restaurant’s description by removing stopwords, punctuation, and stemming words to create a searchable field.
2. **Build the Vocabulary**: Create a vocabulary that maps each unique term in the descriptions to a unique integer `term_id`.
3. **Create an Inverted Index**: Build an inverted index where each `term_id` is associated with a list of document IDs (restaurants) that contain that term.
4. **Implement the Conjunctive Query Function**: Design a function that takes a user query, preprocesses it, retrieves term IDs from the vocabulary, and finds the intersection of document lists in the inverted index to return only documents containing all query terms.
5. **Test and Display Results**: Run sample queries and display the results to verify the functionality of the conjunctive search engine.

This approach will help us retrieve highly relevant restaurants based on the specific combination of terms in a query.

## Vocabulary and Inverted Index Building

In this step, we’ll create the foundation for our search engine by building a **vocabulary** and an **inverted index** from the `processed_description` column of each restaurant.

### Vocabulary
The vocabulary is a dictionary that maps each unique word in the restaurant descriptions to a unique integer ID, called `term_id`. This serves two main purposes:
1. **Uniqueness**: Each word has a single ID, allowing consistent identification across the dataset.
2. **Efficiency**: By using integer IDs instead of words, we reduce memory usage and speed up search operations.

### Inverted Index
The inverted index is a data structure that maps each `term_id` to a list of document IDs (in this case, restaurant indices) that contain the term. This enables us to quickly look up documents containing specific words, which is essential for efficient searching.

For each restaurant’s `processed_description`, we:
1. Split the text into individual terms.
2. Map each term to a `term_id` (using the vocabulary) and assign it a unique integer if it doesn’t already exist in the vocabulary.
3. Add the restaurant’s ID (or index) to the inverted index for each term it contains.

By the end of this step, we’ll have a vocabulary that associates each unique word with a `term_id`, and an inverted index that lets us retrieve restaurants based on these terms.

This setup is critical for enabling efficient **conjunctive queries** later on, as the inverted index lets us intersect document lists for each search term and retrieve only the relevant results.


In [ ]:
from collections import defaultdict

# Initialize vocabulary and inverted index
vocabulary = {}  # maps each term to a unique term_id
inverted_index = defaultdict(list)  # maps each term_id to a list of document IDs (restaurant indices)

# Unique identifier for each term
current_term_id = 0

# Populate vocabulary and inverted index based on the preprocessed descriptions
for doc_id, description in enumerate(df['processed_description']):
    # Split description into terms
    terms = description.split()

    for term in terms:
        # Assign a unique term_id if term is not in vocabulary
        if term not in vocabulary:
            vocabulary[term] = current_term_id
            current_term_id += 1

        # Get the term_id for the current term
        term_id = vocabulary[term]

        # Add the document ID to the inverted index for this term_id
        if doc_id not in inverted_index[term_id]:
            inverted_index[term_id].append(doc_id)

After creating them we check a few conditions to see if they are well built.

In [ ]:
# 1. Check for unique mappings by comparing the vocabulary size to the number of unique terms in processed descriptions.
unique_terms_in_descriptions = set(term for description in df['processed_description'] for term in description.split())
if len(vocabulary) == len(unique_terms_in_descriptions):
    print("Vocabulary integrity check 1: Passed. Each term has a unique mapping.")
else:
    print("Vocabulary integrity check 1: Failed. The vocabulary size does not match the number of unique terms.")

# 2. Check that each term has a unique term_id by verifying no duplicates in the IDs.
unique_ids = set(vocabulary.values())
if len(vocabulary) == len(unique_ids):
    print("Vocabulary integrity check 2: Passed. Each term_id is unique.")
else:
    print("Vocabulary integrity check 2: Failed. Some terms share the same term_id.")

# 3. Verify that each term in the vocabulary appears at least once in any processed description
terms_in_descriptions = {term for description in df['processed_description'] for term in description.split()}
missing_terms = set(vocabulary.keys()) - terms_in_descriptions

if not missing_terms:
    print("Vocabulary integrity check 3: Passed. All terms in the vocabulary are present in descriptions.")
else:
    print(f"Vocabulary integrity check 3: Failed. Terms missing in descriptions: {missing_terms}")

Vocabulary integrity check 1: Passed. Each term has a unique mapping.
Vocabulary integrity check 2: Passed. Each term_id is unique.
Vocabulary integrity check 3: Passed. All terms in the vocabulary are present in descriptions.


In [ ]:
# 1. Check that each term in the vocabulary has an entry in the inverted index.
missing_terms = [term for term, term_id in vocabulary.items() if term_id not in inverted_index]
if not missing_terms:
    print("Inverted index validity check 1: Passed. All vocabulary terms have entries in the inverted index.")
else:
    print(f"Inverted index validity check 1: Failed. Missing terms in the inverted index: {missing_terms}")

# 2. Check that each document ID in the inverted index actually contains the term.
incorrect_mappings = []
for term, term_id in vocabulary.items():
    # Get all document IDs for this term from the inverted index
    doc_ids = inverted_index.get(term_id, [])
    for doc_id in doc_ids:
        # Check if the term is actually present in the document's processed description
        if term not in df['processed_description'].iloc[doc_id].split():
            incorrect_mappings.append((term, doc_id))

if not incorrect_mappings:
    print("Inverted index validity check 2: Passed. All document IDs in the inverted index contain the term.")
else:
    print(f"Inverted index validity check 2: Failed. Incorrect mappings found: {incorrect_mappings}")

# 3. Ensure each document list for a term_id has unique entries (no duplicate document IDs).
duplicate_doc_ids = {}
for term_id, doc_ids in inverted_index.items():
    unique_doc_ids = set(doc_ids)
    if len(unique_doc_ids) != len(doc_ids):
        duplicate_doc_ids[term_id] = [doc_id for doc_id in doc_ids if doc_ids.count(doc_id) > 1]

if not duplicate_doc_ids:
    print("Inverted index validity check 3: Passed. No duplicate document IDs found in any inverted index entry.")
else:
    print(f"Inverted index validity check 3: Failed. Duplicate document IDs found: {duplicate_doc_ids}")

Inverted index validity check 1: Passed. All vocabulary terms have entries in the inverted index.
Inverted index validity check 2: Passed. All document IDs in the inverted index contain the term.
Inverted index validity check 3: Passed. No duplicate document IDs found in any inverted index entry.


## Executing the Conjunctive Query

To execute a conjunctive search query, we’ll follow these steps, which allow the search engine to return only the restaurants that contain **all** specified query terms in their descriptions. This is also known as an “AND search.”

#### Steps

1. **Process the Query**: We preprocess the user’s query using the same method as we used for the descriptions. This includes:
   - Converting text to lowercase.
   - Removing punctuation.
   - Removing stopwords.
   - Applying stemming to reduce words to their root forms.

2. **Retrieve Term IDs**: For each processed term in the query, we look up its unique integer identifier (`term_id`) in the vocabulary. This ID allows us to efficiently search in the inverted index.

3. **Find Matching Documents**: Using the inverted index, we retrieve lists of document IDs that contain each term.

4. **Intersect Document Lists**: Since this is a conjunctive query, we find the intersection of all document lists, ensuring that only documents (restaurants) containing **all** query terms are retained.

5. **Return Results**: For each document ID in the intersection, we retrieve the relevant restaurant details, including:
   - `restaurantName`
   - `address`
   - `description`
   - `website`

### Example
If the user inputs the query `"traditional Italian terrace"`, the search engine will:
   - Process each term.
   - Retrieve and intersect the lists of documents that contain these terms.
   - Output only those restaurants whose descriptions contain all the terms “traditional,” “Italian,” and “terrace.”

This approach is useful for returning highly specific results by ensuring each restaurant in the result list matches the entire query. Run the code to test the query function!

In [ ]:
def preprocess_query(query):
    """Preprocesses the user query."""
    # Apply the same preprocessing as used for the descriptions
    query = query.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = [stemmer.stem(word) for word in query.split() if word not in stop_words]
    return tokens

def execute_conjunctive_query(query):
    """Executes a conjunctive query and returns matching restaurant details."""
    # Preprocess the query terms
    query_terms = preprocess_query(query)

    # Retrieve term IDs for each query term from the vocabulary
    term_ids = [vocabulary[term] for term in query_terms if term in vocabulary]

    # If any term is not in the vocabulary, return no results
    if len(term_ids) != len(query_terms):
        print("Some query terms are not in the vocabulary. No results found.")
        return pd.DataFrame(columns=["restaurantName", "address", "description", "website"])

    # Get the document lists for each term_id from the inverted index
    doc_lists = [inverted_index[term_id] for term_id in term_ids]

    # Find the intersection of all document lists
    matching_doc_ids = list(set(doc_lists[0]).intersection(*doc_lists[1:])) if doc_lists else []

    # Retrieve restaurant details for matching document IDs
    results = df.loc[matching_doc_ids, ["restaurantName", "address", "description", "website"]]
    return results

In [ ]:
# Accept a query input from the user
query = input("RQ2 Conjunctive query: ")

# Run the conjunctive query function using the input query
results = execute_conjunctive_query(query)

results

RQ2 Conjunctive query: traditional Italian terrace


,restaurantName,address,description,website
457,Il Ristorante - Niko Romito,via di Ripetta 73,Situated on the fifth floor of the Hotel Bulga...,https://www.bulgarihotels.com/it_IT/rome/dinin...
1583,La Leggenda dei Frati,costa San Giorgio 6/a,After an uphill approach if you’re arriving he...,https://laleggendadeifrati.it/
1337,Osteria Il Cappello,piazzetta Bruno Lunelli 5,The outdoor terrace at this restaurant overloo...,https://osteriailcappello.it/


### Conjunctive Query Implementation and Testing with Query Examples

To validate the functionality of our conjunctive query search engine, we ran two queries with increasing specificity. This helped us confirm that the search engine correctly filtered restaurants based on multiple terms and adjusted the results accordingly.

#### Example Queries and Reasoning

1. **General Query: "traditional Italian"**
   - **Reason**: This broad query was chosen to test the search engine’s ability to retrieve a wide array of restaurants that feature traditional Italian cuisine. Since these terms are common, we expected the search to return a larger number of results.
   - **Result**: The search engine returned a diverse list of restaurants emphasizing traditional Italian cuisine. This larger output confirmed that the search engine could locate all relevant descriptions containing both "traditional" and "Italian."

2. **Specific Query: "traditional Italian terrace"**
   - **Reason**: By adding the term "terrace," this query became more specific, aiming to filter the results to restaurants that not only feature traditional Italian cuisine but also have a terrace. This helped us test the conjunctive functionality further by adding a distinguishing feature to the initial query.
   - **Result**: The output list was significantly smaller, showing only restaurants that include "traditional," "Italian," and "terrace" in their descriptions. This confirmed that the search engine could accurately refine results as more specific terms are added.

#### Summary

These two queries demonstrated the conjunctive query functionality effectively:
- Broader queries return a larger, more inclusive list of restaurants.
- Adding specific terms narrows down the results to match only restaurants that meet all criteria, allowing for focused, relevant results.

This approach ensures that users can adjust the specificity of their queries to receive either general or highly targeted search results.

## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity

### Ranked Search Engine with TF-IDF and Cosine Similarity

To build a ranked search engine, we will calculate **TF-IDF** scores for each term in each restaurant’s description and use **cosine similarity** to rank results based on their relevance to the query. Below are the main steps for implementation:

#### Steps

1. **Calculate TF-IDF Scores**
   - **TF (Term Frequency)**: For each term in a description, we calculate term frequency, which is the count of the term in that description divided by the total number of terms in the description.
   - **IDF (Inverse Document Frequency)**: Calculate IDF for each term, which measures how unique the term is across all descriptions:
     \begin{equation}
     \text{IDF} = \log\left(\frac{N}{1 + \text{DF}}\right)
     \end{equation}
     where \( N \) is the total number of documents, and \( DF \) is the number of documents containing the term.
   - **TF-IDF**: Multiply TF by IDF for each term in each description to get the final TF-IDF score.

2. **Create the Updated Inverted Index with TF-IDF Scores**
   - Each term in the inverted index will map to a list of tuples. Each tuple will contain a document ID and the TF-IDF score for that term in that document.

3. **Execute the Ranked Query**
   - **Process the Query**: Preprocess the query terms similarly to the descriptions and compute TF-IDF scores for the query terms.
   - **Cosine Similarity**: Calculate cosine similarity between the TF-IDF vector of the query and the TF-IDF vector of each restaurant description.
   - **Ranking**: Sort restaurants by cosine similarity scores and retrieve the top-k results.

The code below will implement steps 1 and 2 by calculating TF-IDF scores for each term and creating the updated inverted index.

## 2.2.1 Inverted Index with TF-IDF Scores

In [ ]:
import numpy as np
from collections import defaultdict
import math

# Step 1: Calculate TF-IDF Scores
N = len(df)  # Total number of documents
term_document_frequency = defaultdict(int)  # DF: Document frequency for each term
tf_idf_index = defaultdict(list)  # Inverted index with TF-IDF scores

# Calculate term frequency (TF) and document frequency (DF) for each term
for doc_id, description in enumerate(df['processed_description']):
    term_counts = defaultdict(int)
    terms = description.split()
    total_terms = len(terms)

    for term in terms:
        term_counts[term] += 1

    for term, count in term_counts.items():
        # TF calculation
        tf = count / total_terms
        term_document_frequency[term] += 1
        # Store TF temporarily in the index for later TF-IDF computation
        tf_idf_index[term].append((doc_id, tf))  # Store TF instead of TF-IDF for now

# Calculate IDF and update the inverted index with TF-IDF scores
for term, doc_list in tf_idf_index.items():
    # IDF calculation
    idf = math.log(N / (1 + term_document_frequency[term]))

    # Update each document's TF with TF-IDF for this term
    tf_idf_index[term] = [(doc_id, tf * idf) for doc_id, tf in doc_list]

# Step 2: Inverted index now has TF-IDF scores

## Sample Verification of the TF-IDF Index

In [ ]:
# Display a few terms and their associated documents with TF-IDF scores
sample_terms = list(tf_idf_index.keys())[:5]  # Adjust the number of terms if needed
sample_tf_idf_index = {term: tf_idf_index[term] for term in sample_terms}

# Print sample
print("Sample of TF-IDF index:")
for term, doc_list in sample_tf_idf_index.items():
    print(f"Term: {term}")
    for doc_id, tf_idf_score in doc_list[:5]:  # Show a limited number of docs per term
        print(f"  Document ID: {doc_id}, TF-IDF Score: {tf_idf_score:.4f}")
    print("-" * 20)

Sample of TF-IDF index:
Term: situat
  Document ID: 0, TF-IDF Score: 0.0268
  Document ID: 3, TF-IDF Score: 0.0479
  Document ID: 7, TF-IDF Score: 0.0479
  Document ID: 13, TF-IDF Score: 0.0263
  Document ID: 32, TF-IDF Score: 0.0463
--------------------
Term: bucol
  Document ID: 0, TF-IDF Score: 0.1129
  Document ID: 521, TF-IDF Score: 0.0733
  Document ID: 573, TF-IDF Score: 0.1344
  Document ID: 708, TF-IDF Score: 0.2258
  Document ID: 1354, TF-IDF Score: 0.0594
--------------------
Term: locat
  Document ID: 0, TF-IDF Score: 0.0510
  Document ID: 6, TF-IDF Score: 0.0340
  Document ID: 10, TF-IDF Score: 0.0849
  Document ID: 12, TF-IDF Score: 0.0214
  Document ID: 14, TF-IDF Score: 0.0392
--------------------
Term: trentino
  Document ID: 0, TF-IDF Score: 0.1079
  Document ID: 219, TF-IDF Score: 0.1079
  Document ID: 1280, TF-IDF Score: 0.1037
  Document ID: 1337, TF-IDF Score: 0.1173
  Document ID: 1541, TF-IDF Score: 0.1254
--------------------
Term: excel
  Document ID: 0, TF-ID

To ensure the accuracy and structure of our TF-IDF index, we extracted a sample of terms with their associated documents and TF-IDF scores. This verification step is important for confirming that:

1. **The Index Structure is Correct**: Each term in the index is linked to a list of document IDs where it appears. Each document ID is paired with the term’s TF-IDF score in that document. This structure allows efficient lookups during search operations.

2. **TF-IDF Scores Appear Reasonable**: TF-IDF scores are generally fractional values, often between 0 and 0.2, with higher values indicating a term that is highly relevant in a specific document. In our sample, values like `0.1129` for "bucol" in document 0 and `0.1079` for "trentino" in document 0 are within an expected range. Higher values for specific terms in select documents indicate that these terms are particularly important in those contexts.

3. **Document Coverage Matches Term Frequency Expectations**: For more common terms (e.g., "situat" and "locat"), we observe links to a broad range of documents, reflecting their general use across the corpus. In contrast, more specific terms (e.g., "bucol" and "trentino") appear in fewer documents, confirming that the index correctly distinguishes between frequent and rare terms.

This sample verification provides confidence that the TF-IDF index has been correctly constructed, with meaningful TF-IDF scores ready to be used in ranking documents by relevance to a given search query.

## 2.2.2 Execute the Ranked Query

### Ranked Query Execution with Cosine Similarity

To implement the ranked query, we will use **cosine similarity** between the query’s TF-IDF vector and each restaurant description’s TF-IDF vector. Here’s the step-by-step breakdown:

1. **Process the Query**: We preprocess the query terms to match the format of our descriptions, applying lowercasing, stopword removal, and stemming.

2. **Construct the Query Vector (TF-IDF)**: We calculate TF-IDF scores for each term in the query:
   - **TF (Term Frequency)**: For each query term, the term frequency is calculated as the term count divided by the total number of terms in the query.
   - **IDF (Inverse Document Frequency)**: We retrieve the IDF values already computed for each term in the vocabulary to calculate the query's TF-IDF scores.

3. **Compute Cosine Similarity**:
   - For each document that contains any of the query terms, we calculate the cosine similarity between the query vector and the document’s TF-IDF vector.
   - Cosine similarity is calculated as the dot product of the two vectors divided by the product of their norms:
     
     \begin{equation}
     \text{Cosine Similarity} = \frac{\text{Dot Product (Query and Document TF-IDF vectors)}}{\text{Norm of Query Vector} \times \text{Norm of Document Vector}}
     \end{equation}

   - This score provides a measure of relevance between the query and each document, with values ranging from 0 (no similarity) to 1 (identical vectors).

4. **Sort and Return Top-k Results**:
   - Sort the documents by cosine similarity score in descending order.
   - Retrieve the top-k results, each with the restaurant's `name`, `address`, `description`, `website`, and similarity score.

The code below implements these steps, retrieving and ranking restaurants based on the relevance of their descriptions to the query.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def execute_ranked_query(query, k=5):

    # Preprocess the query using the same preprocessing as the dataset
    preprocessed_query = preprocess_description(query)
    description_col = df['processed_description']

    # Vectorize the preprocessed query
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(description_col)
    query_vector = vectorizer.transform([preprocessed_query])

    # Compute cosine similarity between the query and all restaurant vectors
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Get the top-k indices sorted by similarity score
    top_indices = np.argsort(cosine_similarities)[::-1][:k]

    # Filter results with non-zero similarity
    top_indices = [i for i in top_indices if cosine_similarities[i] > 0]

    # Prepare the output
    results = df.loc[top_indices, ['restaurantName', 'address', 'description', 'website']].copy()
    results['Similarity Score'] = cosine_similarities[top_indices]
    results = results.sort_values(by='Similarity Score', ascending=False)

    return results

In [ ]:
# Accept a query input from the user
query = input("RQ2 Ranked query: ")

# Run the conjunctive query function using the input query
results = execute_ranked_query(query)

results

RQ2 Ranked query: traditional Italian terrace


,restaurantName,address,description,website,Similarity Score
1098,ZELO,via Gesù 6/8,The elegant ZELO restaurant enjoys a beautiful...,https://www.fourseasons.com/it/milan/dining/re...,0.237762
1866,La Veranda,via Regina 40,"As its name suggests, the dining room at this ...",https://www.villadeste.com/it/hotel-con-ristor...,0.219165
1833,Caffè Dante Bistrot,piazza dei Signori 2,This restaurant boasts an outdoor terrace over...,https://www.caffedante.it/,0.196841
924,Il Pozzo,viale Allegri 7,This restaurant is housed in a historic palazz...,NaN,0.194572
261,Zàghara Restaurant,Contrada Bigini snc,"Housed in the elegant Relais Villa Flora, this...",http://www.villaflorarelais.it,0.189553


### Explanation of Results

We followed the prescribed method, including:

1. **Preprocessing**:
   - Applied consistent preprocessing to both the dataset and the query, including lemmatization, removal of punctuation, stopwords, and converting text to lowercase.
   - Ensured uniformity to maximize the alignment between the query and the dataset.

2. **TF-IDF Vectorization**:
   - Used a `TfidfVectorizer` with optimized parameters such as `ngram_range=(1,2)` to include bigrams, `max_df=0.85`, and `min_df=2`, to filter out overly common or rare terms.

Despite these efforts, the cosine similarity values remain relatively small (below 0.2). This can be attributed to several factors:

- **Dataset Specificity**: The dataset may not have detailed or highly descriptive text that directly aligns with the query. For example, if the descriptions do not frequently mention "modern" or "seasonal cuisine" explicitly, the cosine similarity values will naturally be lower.
  
- **Query Specificity**: The query "modern seasonal cuisine" is quite specific, and if the dataset descriptions are generic, they might not capture the full context of the query.

- **Limitations of TF-IDF**:
   - TF-IDF relies on exact term matching and does not understand semantic relationships (e.g., "modern" and "contemporary" are semantically similar but treated as different terms).
   - Even with bigrams, the matching depends on the exact occurrence of the phrase "modern cuisine" in the text.

In conclusion, while the results are meaningful in ranking the dataset entries, the relatively low cosine similarity values are a known limitation of the TF-IDF approach when applied to datasets with sparse or generic textual descriptions.

# RQ3. Define a New Score!

Now, we will define a custom ranking metric to prioritize restaurants based on user queries.

Steps:

1. User Query: The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.

2. New Ranking Metric: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the description field, we can include other attributes like `priceRange`, `facilitiesServices`, and `cuisineType`.

3. You will use a heap data structure (e.g., Python’s heapq library) to maintain the top-k restaurants.

4. Are the results you obtain better than with the previous scoring function? Explain and compare results.

#### STEP 1: User query

User's input is preprocessed by using the `preprocessed_query()` function which does the following:
   - Converts text to lowercase.
   - Removes punctuation.
   - Removes stopwords.
   - Applies stemming to reduce words to their root forms.

This preprocessing is important as it allows for correct comparision between the query and characteristics of each restaurant.
In the next step, we will apply `execute_conjunctive_query()` function to the query in order to retrieve matching documents.
   As we are interested in additional information that are not returned by the
`execute_conjunctive_query()` function we can add this columns from the prepared subset_df by simple left-join merge.

In [ ]:
import heapq

# Make subset with the relevant data for scoring function
subset_df = df[['restaurantName','priceRange', 'facilitiesServices', 'cuisineType', 'processed_description']]
print(subset_df.head())

             restaurantName priceRange  \
0            Castel Toblino         €€   
1               Su Gologone         €€   
2           Bàcaro Il Gusto         €€   
3  Osteria dello Strecciolo        €€€   
4                    Trippa         €€   

                                  facilitiesServices  \
0          ['Car park', 'Garden or park', 'Terrace']   
1  ['Air conditioning', 'Car park', 'Garden or pa...   
2          ['Air conditioning', 'Wheelchair access']   
3                                                 []   
4  ['Air conditioning', 'Terrace', 'Wheelchair ac...   

                      cuisineType  \
0    Modern Cuisine, Contemporary   
1  Sardinian, Traditional Cuisine   
2                        Venetian   
3            Italian Contemporary   
4       Italian, Seasonal Cuisine   

                               processed_description  
0  situat bucol locat trentino excel vino santo p...  
1  difficult choos three delight room restaur one...  
2  simpl modern restau

In [ ]:
# Query input and preprocessing
query = input('RQ3 New score query:')
processed_query = preprocess_query(query)

# Convert the list into string for downstream tf-idf
processed_query_str = ' '.join(processed_query)

# Dataframe of the search results
# This function has buit into it the query preprocessing so we simply use the unprocessed query
search_results_df = execute_conjunctive_query(query)

# Print to visually inspect the matched documents preprocessed query
print('Preprocessed query:', processed_query_str)
print(search_results_df.head())

RQ3 New score query:traditional italian terrace
Preprocessed query: tradit italian terrac
                   restaurantName                    address  \
457   Il Ristorante - Niko Romito          via di Ripetta 73   
1583        La Leggenda dei Frati      costa San Giorgio 6/a   
1337          Osteria Il Cappello  piazzetta Bruno Lunelli 5   

                                            description  \
457   Situated on the fifth floor of the Hotel Bulga...   
1583  After an uphill approach if you’re arriving he...   
1337  The outdoor terrace at this restaurant overloo...   

                                                website  
457   https://www.bulgarihotels.com/it_IT/rome/dinin...  
1583                     https://laleggendadeifrati.it/  
1337                      https://osteriailcappello.it/  


In [ ]:
# Complete data frame composed of the search results merged with the scoring function-relevant columns
results_complete_df = pd.merge(search_results_df, subset_df, on='restaurantName', how='left')
print(results_complete_df.head())

                restaurantName                    address  \
0  Il Ristorante - Niko Romito          via di Ripetta 73   
1  Il Ristorante - Niko Romito          via di Ripetta 73   
2        La Leggenda dei Frati      costa San Giorgio 6/a   
3          Osteria Il Cappello  piazzetta Bruno Lunelli 5   

                                         description  \
0  Situated on the fifth floor of the Hotel Bulga...   
1  Situated on the fifth floor of the Hotel Bulga...   
2  After an uphill approach if you’re arriving he...   
3  The outdoor terrace at this restaurant overloo...   

                                             website priceRange  \
0  https://www.bulgarihotels.com/it_IT/rome/dinin...       €€€€   
1  https://www.bulgarihotels.com/it_IT/rome/dinin...       €€€€   
2                     https://laleggendadeifrati.it/       €€€€   
3                      https://osteriailcappello.it/         €€   

                                  facilitiesServices  \
0  ['Air conditioning

#### STEP 2: Define a scoring function that takes into account various attributes:

**TF-IDF Computation**:
TF_IDF is a metric that evaluates the importance of a word in a document with respect to the whole collection of documents. In this way it identifies the key terms of a particular document. We will use this metric by first running the vectorizer on the `processed_description` column so it learn the vocabulary and scores it and followingly on the query itself.

**Cosine Similarity**:
A metric used to evaluate the similarity of two vectors, by computing the cosine angle between them (1 for identical, 0 for orthogonal). Here we will apply it to the previously computed query vector and the document matrix. A result will be the score of how similar the query is to each of the retrieved restaurant descriptions.

In [ ]:
# Compute the TF-IDF and add them as an additional column
description_col = results_complete_df['processed_description']
vectorizer = TfidfVectorizer()

# Fit and transform the processed_description column and processed query
tfidf_matrix = vectorizer.fit_transform(description_col)
tfidf_query = vectorizer.transform([processed_query_str])

# Compute cosine similarity between the query and the processed_description column
similarity_scores = cosine_similarity(tfidf_query, tfidf_matrix)
results_complete_df["cosineSimilarity"] = similarity_scores.flatten()

# Display the DataFrame
print(results_complete_df.to_string())

                restaurantName                    address                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            description                        

##### New scoring function:
- **Description Match**: Give weight based on the query similarity to the description (using TF-IDF scores).
- **Cuisine Match**: Increase the score for matching cuisine types.
- **Facilities and Services**: Give more points for matching facilities/services (e.g., “Terrace,” “Air conditioning”).
- **Price Range**: Higher scores could be given to more affordable options based on the user’s choice.

Considerations:

- **Scoring**: As the "*restaur*" term appears quite often in the searches and so does in the `facilitiesServices` but does not aport any additional value, it will be excluded from the scoring.

In [ ]:
# Total scoring function and helper match functions


# Function that scores query match with the cuisine type
def cuisine_match(query, row):
    score = 0
    for term in query:
        if term in preprocess_description(row['cuisineType']):
            score = 1
    return score

# Function that scores the query match with the facilities and services
def facility_match(query, row):
    score = 0
    for term in query:
        if term in preprocess_description(row['facilitiesServices']):
            if term != 'restaur':
                score += 1
    return score

# Function for price score
def price_match(row):
    prices = {'€€€€':0,'€€€':1,'€€':2,'€':3}
    score = prices.get(row['priceRange'])
    return score

# Function computing the overall score for every restaurant using the above defined match functions
def new_attribute_score(query, row, cuisine_weight=0.5, facility_weight=0.3, price_weight=0.2):
    cuisine_score = cuisine_match(query, row)
    facility_score = facility_match(query, row)
    price_score = price_match(row)
    similarity = row['cosineSimilarity']

    total_score = similarity*(cuisine_score*cuisine_weight + facility_score*facility_weight + price_score*price_weight +1)
    return(total_score)

#### STEP 3: Top k restaurants: You will use a heap data structure (e.g., Python’s heapq library) to maintain the top-k restaurants.

The output should include:
- restaurantName
- address
- description
- website
- the new similarity score based on the custom metric


To create and maintain a heap of the top restaurants based on the `newScore` function, we use the `top_k_restaurants()` function that takes as input the search results, the query and a variable k, representing the number of top restaurants we want in the output.

FUNCTION DESCRIPTION

 The function creates an empty heap and then proceeds to iterate over the rows of the search results assigning a score to each restaurant considering the matches with `priceRange`, `facilitiesServices` and `cuisineType`. Based on the score it then adds it to the heap which if completely filled will push out the restaurant with lowest score.
    Followingly, a simple dictionary of the top restaurant is created and used to filter out
the relevant information for the output from the search result dataframe.
    Lastly, data frame of the top k restaurants in descending order based on the newScore
is returned.

In [ ]:
# Top restaurants function using heap as data structure
def top_k_restaurants(df, query, k):

    if df.empty:
        return 'We found no result to your query.'
    else:
        heap = []
        # Assign a score to each restaurant by iterating over the data frame
        for index, row in df.iterrows():
            score = new_attribute_score(query,row)
            restaurant = row['restaurantName']

            if len(heap) < k:
                heapq.heappush(heap,(score, restaurant))
            else:
                heapq.heappushpop(heap, (score, restaurant))

        # Dictionary with the restaurant name and scores
        top_restaurants_dict = {name:score for score, name in heap}

        # Subset the dataframe with the top restaurants and get selcted columns
        top_restaurants_df = df[df['restaurantName'].isin(top_restaurants_dict.keys())][['restaurantName','address','cuisineType','priceRange','website', 'Latitude', 'Longitude', 'Region']]
        top_restaurants_df['newScore'] = top_restaurants_df['restaurantName'].map(top_restaurants_dict)

        # Sort the top restaurants in descending order based
        top_restaurants_df = top_restaurants_df.sort_values(by='newScore', ascending=False)

        return top_restaurants_df

# Function call to get k top restaurants based on the newScore
top_restaurants = top_k_restaurants(results_complete_df, processed_query, k=10)
print(top_restaurants)

                restaurantName                    address  \
3          Osteria Il Cappello  piazzetta Bruno Lunelli 5   
0  Il Ristorante - Niko Romito          via di Ripetta 73   
1  Il Ristorante - Niko Romito          via di Ripetta 73   
2        La Leggenda dei Frati      costa San Giorgio 6/a   

                cuisineType priceRange  \
3  Classic Cuisine, Italian         €€   
0      Italian Contemporary       €€€€   
1      Italian Contemporary       €€€€   
2                  Creative       €€€€   

                                             website  newScore  
3                      https://osteriailcappello.it/  0.381795  
0  https://www.bulgarihotels.com/it_IT/rome/dinin...  0.325036  
1  https://www.bulgarihotels.com/it_IT/rome/dinin...  0.325036  
2                     https://laleggendadeifrati.it/  0.151703  


#### STEP 4: Compare the results
The new scoring function is more sensitive as it takes into account other parameters than just the similarity between description and the query. Adding query-dependent facility score and cuisine score improves the results especially in cases when these might not be mentioned in the description. On the other hand, the price score fixed for each restaurant based on the `priceRange` ensures the preofrence will be added to the budget-friendlier restaurants. Finally, having the option of setting the weights for each of these parameters, the user can opt to exclude any of these from their search or taylor them to their needs.

# RQ4. Visualizing the Most Relevant Restaurants

We had to first compile distinct locations in the form of cities and regions in order to finish this step. Using OpenCage's API, we obtained the required data. We created a CSV file with the geographic data for every restaurant using its latitude, longitude and region.

In [ ]:
!pip install folium

# Use query and k for filtering
query_terms = ["italian", "cuisine"]  # Example query terms
k = 5  # Number of top restaurants

top_restaurants = top_k_restaurants(results_complete_df, processed_query, k)

import folium
from folium.plugins import MarkerCluster

# Initialize a map centered around the average latitude and longitude of the dataset
center_lat = top_restaurants['Latitude'].mean()
center_lon = top_restaurants['Longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Define price range colors
price_colors = {
    '€': 'green',
    '€€': 'blue',
    '€€€': 'orange',
    '€€€€': 'red'
}

# Add a marker cluster to group nearby restaurants
marker_cluster = MarkerCluster().add_to(m)

# Add restaurants to the map
for _, row in top_restaurants.iterrows():
    marker_color = price_colors.get(row['priceRange'], 'gray')  # Default to gray if price range not found
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=8,  # Fixed marker size for visibility
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.7,
        popup=(
            f"<b>Name:</b> {row['restaurantName']}<br>"
            f"<b>Region:</b> {row['Region']}<br>"
            f"<b>Price Range:</b> {row['priceRange']}<br>"
            f"<b>Address:</b> {row['address']}<br>"
            f"<b>Website:</b> <a href='{row['website']}' target='_blank'>{row['website']}</a>"
        )
    ).add_to(marker_cluster)

# Add a legend to the map
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 200px; height: 150px;
            background-color: white; z-index:9999; font-size:14px;
            border:2px solid grey; padding: 10px;">
    <b>Price Range Legend</b><br>
    <i style="background:green; color:green; border-radius:50%; padding:5px;"></i> €<br>
    <i style="background:blue; color:blue; border-radius:50%; padding:5px;"></i> €€<br>
    <i style="background:orange; color:orange; border-radius:50%; padding:5px;"></i> €€€<br>
    <i style="background:red; color:red; border-radius:50%; padding:5px;"></i> €€€€<br>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file and display top 5 restaurants as an instance
map_file = 'top_restaurants_map.html'
m.save(map_file)

map_file

KeyError: 'Latitude'

In [ ]:
from IPython.core.display import HTML
HTML('<img src="top_restaurants_map.html" alt="Top restaurants map" width="400">')


In [ ]:
from IPython.display import IFrame

# Display the uploaded HTML file
IFrame(src='top_5_restaurants_map.html', width=800, height=600)

# RQ5. BONUS: Advanced Search Engine

In [ ]:
# Function to create an inverted index for fast search on selected fields
def create_inverted_index(data):
    index = defaultdict(set)

    # Tokenize and index fields for search with separate handling for fields and words
    for i, row in data.iterrows():

        for field in ["restaurantName", "city", "cuisineType"]:
            field_value = row.get(field)            # Retrieve the value of the field for the current row
            if pd.notna(field_value):               # Check if the value is not NaN
                words = field_value.lower().split() # Tokenize the value into words

                for word in words:
                    if word not in index:
                        index[word] = set()         # Initialize a new set if the word is not in the index
                    index[word].add(i)              # Add the index of the row to the set for this word
    return index

In [ ]:
# Function to search the restaurants
def restaurants_search(data, name=None, city=None, cuisine_type=None, min_price=None, max_price=None, regions=None, credit_cards=None, facilities=None):

    # Create a copy of data
    temporary_data = data.copy()

    # Create an inverted index to do efficient search
    index = create_inverted_index(temporary_data)

    matching_indices = set(temporary_data.index)

    # Name's search logic function
    if name:

        # Find all matches for name terms using the inverted index
        name_matches = set.union(*(index.get(term, set()) for term in name.lower().split()))
        matching_indices = matching_indices.intersection(name_matches)

    # City's search logic function
    if city:

        # Find all matches for city terms using the inverted index
        city_matches = set.union(*(index.get(term, set()) for term in city.lower().split()))
        matching_indices = matching_indices.intersection(city_matches)

    # Cuisine type's search logic function
    if cuisine_type:

        # Find all matches for cuisine type terms using the inverted index
        cuisine_matches = set.union(*(index.get(term, set()) for term in cuisine_type.lower().split()))
        matching_indices = matching_indices.intersection(cuisine_matches)

    # Price range's filter
    if min_price or max_price:

        # Map of price symbols to numerical values
        price_values = {"€": 1, "€€": 2, "€€€": 3, "€€€€": 4}
        min_price_value = price_values.get(min_price, 1)  # Default to 1
        max_price_value = price_values.get(max_price, 4)  # Default to 4

        # Filter indices based on price range
        price_matches = {idx for idx in temporary_data.index if price_values.get(temporary_data.at[idx, "priceRange"], 0) >= min_price_value and price_values.get(temporary_data.at[idx, "priceRange"], 0) <= max_price_value}
        matching_indices = matching_indices.intersection(price_matches)

    # Region's filter
    if regions:

        # Postal codes and their ranges (with the corresponding regions)
        postal_code_ranges = [
            (100, 199, "Lazio"), (1000, 1999, "Lazio"), (3000, 3999, "Lazio"),
            (7000, 9199, "Sardegna"), (10000, 12999, "Piemonte"),
            (13000, 13999, "Valle d'Aosta"), (15000, 19999, "Piemonte"),
            (20000, 20999, "Lombardia"), (21000, 23999, "Lombardia"),
            (25000, 28999, "Lombardia"), (30000, 34999, "Veneto"),
            (36000, 36999, "Veneto"), (37000, 39999, "Veneto"),
            (38000, 38999, "Trentino-Alto Adige"), (40000, 47999, "Emilia-Romagna"),
            (50000, 59100, "Toscana"), (60000, 63999, "Marche"),
            (64000, 64999, "Abruzzo"), (66000, 66999, "Abruzzo"),
            (67000, 67999, "Molise"), (70000, 71999, "Puglia"),
            (75000, 75999, "Basilicata"), (80000, 84999, "Campania"),
            (85000, 86999, "Basilicata"), (87000, 89999, "Calabria"),
            (90000, 91999, "Sicilia"), (95000, 98199, "Sicilia")
        ]

        # Function to determine the region dependent on the postal code
        def region_from_postal_code(postal_code):
            try:
                postal_code = int(postal_code)
                matched_region = next((region for start, end, region in postal_code_ranges if start <= postal_code <= end), "Unknown Region")
                return matched_region

            except (ValueError, TypeError):
                return "Unknown Region"

        # Add the region column to temporary_data
        temporary_data["region"] = temporary_data["postalCode"].apply(region_from_postal_code).str.lower()

        # Apply region filter
        region_list = {region.strip().lower() for region in regions.split(",")}
        region_matches = {idx for idx in temporary_data.index if temporary_data.at[idx, "region"] in region_list}
        matching_indices = matching_indices.intersection(region_matches)

    # Credit cards' filter
    if credit_cards:

        # Filter indices based on presence of specified credit cards
        credit_matches = {idx for idx in temporary_data.index if any(card in temporary_data.at[idx, "creditCards"] for card in credit_cards)}
        matching_indices = matching_indices.intersection(credit_matches)

    # Facilities' filter
    if facilities:

        # Filter indices based on presence of specified facilities
        facility_matches = {idx for idx in temporary_data.index if all(facility in temporary_data.at[idx, "facilitiesServices"] for facility in facilities)}
        matching_indices = matching_indices.intersection(facility_matches)

    # Filtered data
    result = temporary_data.loc[list(matching_indices), ["restaurantName", "address", "city", "cuisineType", "priceRange", "region", "website"]]

    return result

# Algorithmic Question (AQ)

1. Write the pseudocode for an algorithm that solves this problem.

In [ ]:
def solve():
    T = int(input()) # Number of test cases

    for t in range(T):
        n = int(input())  # Number of packages
        packages = []  # Package coordinates

        for i in range(n):
            x, y = list(map(int, input().split())) # x and y coordinates of a package
            packages.append((x, y))

        # Sort packages by (x, y) coordinates
        packages.sort()  # Sorts first by x, then by y (if x's are the same)

        # Check if a valid path exists
        possible = True
        for i in range(1, n):
            # Check if coordinates of the current package is less than coordinates of the previous package
            if packages[i][0] < packages[i - 1][0] or packages[i][1] < packages[i - 1][1]:
                possible = False
                break

        if not possible:
            print("NO")
            continue

        # Construct the path if possible
        current_x, current_y = 0, 0
        path = ""

        for x, y in packages:
            # Move right (R) to reach the target x-coordinate
            path += "R" * (x - current_x)
            # Move up (U) to reach the target y-coordinate
            path += "U" * (y - current_y)
            # Update current position
            current_x, current_y = x, y

        # Output the result
        print("YES")
        print(path)
solve()

3
5
1 3
1 2
3 3
5 5 
4 3
YES
RUUURRRRUU
2 
1 0
0 1
NO
1
4 3
YES
RRRRUUU


2. Prove that your algorithm is correct.

It can be proved that the algorithm is correct by establishing an invariant — a condition that holds true at each step of the algorithm.

#### Invariant 1: Sorted Order Implies Lexicographical Order
**Claim**: Sorting the packages by `(x, y)` coordinates ensures that visiting them in this order produces the lexicographically smallest path.

**Proof**: Sorting by `(x, y)` ensures that if two packages have the same `x` coordinate, the one with the smaller `y` coordinate appears first. This ordering respects the lexicographical priority of moving right (`R`) before moving up (`U`). As a result, if there’s a valid path that collects all packages in this order, it will be the lexicographically smallest.

#### Invariant 2: Reachability Check Guarantees Feasibility
**Claim**: If for every pair of consecutive packages `(xᵢ, yᵢ)` and `(xᵢ₊₁, yᵢ₊₁)` in the sorted list, `xᵢ ≤ xᵢ₊₁` and `yᵢ ≤ yᵢ₊₁`, then all packages are reachable using only right and up moves.

**Proof**: If `xᵢ > xᵢ₊₁` or `yᵢ > yᵢ₊₁` for any two consecutive packages, it would mean moving left or down is required to reach the next package, which violates the movement restrictions. Therefore, the algorithm checks each pair of consecutive packages after sorting to ensure `x` and `y` coordinates are non-decreasing. If this holds for all pairs, then it is guaranteed that we can move from each package to the next using only right or up moves.

#### Invariant 3: Constructed Path is Minimal
**Claim**: The constructed path is the minimal (shortest) path to collect all packages in the lexicographically smallest order.

**Proof**: After confirming that each package is reachable in the sorted order, the algorithm constructs the path by calculating the difference in `x` (right moves) and `y` (up moves) between the current position and each package. Since the algorithm adds all `R` moves before `U` moves for each step, this minimizes unnecessary movement and maintains lexicographical order. Thus, this approach yields the shortest path that respects the lexicographical requirement.

3. Compute the time complexity of your algorithm in Big O notation. Break down the steps involved in the algorithm, and explain which parts contribute most to the overall time complexity.

1) Reading Input for Each Test Case:

In [ ]:
T = int(input()) # Number of test cases

for t in range(1, T + 1):
    n = int(input())  # Number of packages
    packages = []

    for i in range(n):
        x, y = list(map(int,input()))
        packages.append((x, y))

This part reads **T (the number of test cases)** and, for each test case, reads **n packages** with their coordinates.

Time complexity:
* Reading each integer takes O(1), so reading n packages per test case takes O(n)
* For T test cases, the total time complexity for reading input is: O(T * n)

2) Sorting the Packages

In [ ]:
packages.sort()

The packages are sorted by coordinates (x, y), which helps in constructing the lexicographically smallest path.

Time complexity:
* Sorting n packages takes O(*n* log *n*). It requires about log *n* levels of division to fully sort the list, and at each level, comparing *n* elements requires O(n) time.
* For T test cases, the time complexity for sorting is: O(T * *n* log *n*)

3) Verifying Reachability

In [ ]:
possible = True
for i in range(1, n):
    if packages[i][0] < packages[i - 1][0] or packages[i][1] < packages[i - 1][1]:
        possible = False
        break

This loop checks if all packages can be reached using only right (R) and up (U) moves. If any package cannot be reached in this way, it sets possible to False.

Time complexity:
* This loop iterates through the sorted list of n packages. Across T test cases, the complexity for verifying reachability is: O(T * n)

4) Constructing the Path

In [ ]:
current_x, current_y = 0, 0
path = ""

for (x, y) in packages:
    path += "R" * (x - current_x)
    path += "U" * (y - current_y)
    current_x, current_y = x, y

If all packages are reachable, this step constructs the path by iterating through each package and calculating the required number of R and U moves.

Time complexity:
* Total moves per package are bounded by the grid size n. Across T test cases, the total complexity for path construction is: O(T * n)

The most computationally expensive step is sorting the packages, which has a time complexity of **O(T * *n* log *n*)**. Therefore, the overall time complexity of the algorithm is **O(T * *n* log *n*)**.

4. Ask an LLM tool (such as ChatGPT, Claude AI, Gemini, Perplexity, etc.) to evaluate the time complexity of your code using Big O notation. Is the assessment accurate? If it differs from your previous analysis, which would be correct? Please explain your reasoning.

### Time Complexity Analysis of `solve()` Function

To analyze the time complexity of the code, we’ll examine each part of the function `solve()`.

#### 1. Reading Input
- `T = int(input())` takes constant time, $O(1)$, to read the number of test cases.
- For each test case, `n = int(input())` reads the number of packages. Assuming each integer read takes constant time, $O(1)$, this step itself is $O(T)$.

#### 2. Loop Through Test Cases
- The main loop iterates $T$ times (once per test case), so anything inside this loop is performed $T$ times.

#### 3. Reading and Storing Coordinates
- For each test case, the loop `for i in range(n):` iterates $n$ times, where each iteration reads a coordinate pair `(x, y)` and appends it to the `packages` list.
- The complexity of reading all coordinates for a test case is $O(n)$, assuming each read is $O(1)$.

#### 4. Sorting the Packages
- After reading all packages, the code sorts the list of packages by their `(x, y)` coordinates.
- Sorting $n$ elements takes $O(n \log n)$ time. This is the most computationally expensive operation within a test case.

#### 5. Checking Validity of Path
- The next loop, `for i in range(1, n):`, checks if each package’s coordinates are non-decreasing. This loop runs $n-1$ times.
- Each iteration involves a constant-time comparison, making the complexity of this loop $O(n)$.

#### 6. Constructing the Path
- If the path is valid, the final loop constructs the path by moving horizontally and vertically between package coordinates.
- This loop iterates $n$ times, and each iteration appends characters to the `path` string.
- In the worst case, if we have to move across a large distance (e.g., from `(0, 0)` to `(x, y)`), we would append $x + y$ characters to the string.
- While the exact number of moves depends on the input coordinates, in the worst case, we can assume constructing the path would take $O(n)$ time for each test case.

#### 7. Printing the Result
- Printing the result is $O(1)$ for each test case, as it outputs constant-size strings.

### Total Time Complexity
Combining all these steps, the overall time complexity for each test case is dominated by the sorting step, which is $O(n \log n)$. Thus, for $T$ test cases, the total time complexity of the function is:

$$
O(T \cdot (n \log n + n)) = O(T \cdot n \log n)
$$

### Final Complexity
The time complexity of the function `solve()` is:

$$
O(T \cdot n \log n)
$$

where:
- $T$ is the number of test cases.
- $n$ is the number of packages (coordinates) in each test case.

5. Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.

- Let the robot start at position $P_0$, and let there be $n$ packages located at positions $P_1, P_2, \dots, P_n $.
- Let $d(A, B)$ denote the distance between points $A$ and $B$, calculated as the Manhattan distance:

  $
  d(A, B) = |x_A - x_B| + |y_A - y_B|
  $

### Greedy Algorithm

1. At each step, from the robot’s current position $P_i$, the greedy algorithm moves to the nearest remaining package $P_{i+1}$ such that:

   $
   d(P_i, P_{i+1}) = \min_{P_k \in \text{Remaining Packages}} d(P_i, P_k)
   $

2. The total distance $D_{\text{greedy}}$ traveled by the robot following the greedy algorithm is:

   $
   D_{\text{greedy}} = \sum_{i=0}^{n-1} d(P_i, P_{i+1})
   $



To prove that the greedy algorithm is optimal, we use the **triangle inequality**, which states that for any three points $A$, $B$, and $C$:
$
d(A, B) \leq d(A, C) + d(C, B)
$


### Proof by Contradiction

Assume, for contradiction, that there exists a different path, $D_{\text{alt}}$, that is shorter than the greedy path $D_{\text{greedy}}$:
$
D_{\text{alt}} < D_{\text{greedy}}
$
Let this alternative path visit packages in a non-greedy order.

1. **Alternative Path Analysis**:
   - In $D_{\text{alt}}$, suppose the robot skips a closer package $P_j$ in favor of a farther package $P_k$ (where $d(P_i, P_j) < d(P_i, P_k)$), and then returns to visit $P_j$ later.
   - By the triangle inequality:
     $
     d(P_i, P_j) \leq d(P_i, P_k) + d(P_k, P_j)
     $
   - Therefore, going directly from $P_i$ to $P_j$ is shorter than or equal to first going to $P_k$ and then to $P_j$.

2. **Contradiction**:
   - By skipping the closest package $P_j$ and going to a farther one $P_k$, $D_{\text{alt}}$ must be longer than or equal to $D_{\text{greedy}}$.
   - This contradicts our assumption that $D_{\text{alt}} < D_{\text{greedy}}$.


Therefore, the greedy algorithm is optimal.